In [7]:
%load_ext autoreload
%autoreload 2

import pickle
import cloudpickle
import pandas as pd

import deploy_model as dm
import predict as predict_script
import numerapi


def unpickle(fl):
    with open(fl, "rb") as infile:
        return pickle.load(infile)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#!aws s3 cp --recursive s3://numerai-v1/experiments/train_on_all_data_2023-05-01_16h-10m/models/ ./

## 1. Download and build ensemble model

In [3]:
argn_mdls = {
    tgt: unpickle(fl=f"./model_{tgt}_v4_20.pkl.pkl")
    for tgt in ["cyrus", "tyler", "ben", "waldo", "victor", "nomi"]
}
argn_mdls["tyler"]

LGBMRegressor(colsample_bytree=0.2, learning_rate=0.002, max_depth=6,
              n_estimators=20000, num_leaves=64)

In [ ]:
argentina_no_ntr = dm.EnsembleNeutralModel(
    models=argn_mdls,
    neutralisation_cols=None,
    neutralisation_prop=None,
    ensembling_fn=dm.argentina_ensemble,
)

## 2. Sanity check it works with both int8 and float32 features

### 2.1 Script using int8 data

In [4]:

# Loads int8 data
pred_arg_nontr_df = predict_script.predict(napi=numerapi.NumerAPI(), wrapped_model=argentina_no_ntr)
pred_arg_nontr_df

[2023-06-09 02:28:09,280] INFO - reading prediction data
[2023-06-09 02:28:10,219] INFO - target file already exists
[2023-06-09 02:28:10,221] INFO - download complete
[2023-06-09 02:28:10,226] INFO - Downloaded live data to v4.1/live_502.parquet...
[2023-06-09 02:28:10,717] INFO - generating predictions


Predicting for each model: 100%|█| 6/6 [00:21<00:00,  3.6

[2023-06-09 02:28:32,893] INFO - Ensembling predictions with argentina_ensemble(): ['pred_cyrus', 'pred_tyler', 'pred_ben', 'pred_waldo', 'pred_victor', 'pred_nomi']
[2023-06-09 02:28:32,899] INFO - Taking the rank percent


,prediction
id,
n000124edbee5931,0.840941
n0006fd05e5c5171,0.854737
n0008241720e02e0,0.102658
n002cecd72ff9453,0.095354
n0033b5ad4d2f9a0,0.730980
...,...
nffcf4bdcf971590,0.936093
nffd07e017f3def4,0.480219
nfff296ce15d1d13,0.683912


### 2.2 Load float data and verify model predicts identical results

In [9]:
napi = numerapi.NumerAPI()
napi.download_dataset("v4.1/live.parquet", "live.parquet")
argentina_no_ntr.predict(pd.read_parquet("live.parquet"))

[2023-06-09 02:32:34,878] INFO - target file already exists
[2023-06-09 02:32:34,883] INFO - starting download


live.parquet: 7.85MB [00:00, 11.5MB/s]                   
Predicting for each model: 100%|█| 6/6 [00:21<00:00,  3.6

[2023-06-09 02:32:58,039] INFO - Ensembling predictions with argentina_ensemble(): ['pred_cyrus', 'pred_tyler', 'pred_ben', 'pred_waldo', 'pred_victor', 'pred_nomi']
[2023-06-09 02:32:58,042] INFO - Taking the rank percent


,prediction
id,
n000124edbee5931,0.840941
n0006fd05e5c5171,0.854737
n0008241720e02e0,0.102658
n002cecd72ff9453,0.095354
n0033b5ad4d2f9a0,0.730980
...,...
nffcf4bdcf971590,0.936093
nffd07e017f3def4,0.480219
nfff296ce15d1d13,0.683912


## 3. Cloudpickle predict file for pickle uploads

In [11]:
pkl_fl = "./models/pkl_upload_argentina_no_ntr.pkl"
with open(pkl_fl, "wb") as outfile:
    cloudpickle.dump(obj=argentina_no_ntr.predict, file=outfile)

### Unpickle and verify predictions w float32 for pickle uploads

In [14]:
arg_predict_from_pkl = unpickle(pkl_fl)

In [15]:
arg_predict_from_pkl(pd.read_parquet("live.parquet"))

Predicting for each model: 100%|█| 6/6 [00:23<00:00,  3.8

[2023-06-09 02:38:23,309] INFO - Ensembling predictions with argentina_ensemble(): ['pred_cyrus', 'pred_tyler', 'pred_ben', 'pred_waldo', 'pred_victor', 'pred_nomi']
[2023-06-09 02:38:23,314] INFO - Taking the rank percent


,prediction
id,
n000124edbee5931,0.840941
n0006fd05e5c5171,0.854737
n0008241720e02e0,0.102658
n002cecd72ff9453,0.095354
n0033b5ad4d2f9a0,0.730980
...,...
nffcf4bdcf971590,0.936093
nffd07e017f3def4,0.480219
nfff296ce15d1d13,0.683912
